In [1]:
import struct
import numpy as np
import sys

# Variable Codes

In [4]:
def encode(lst):
    def encode_n(number):
        btarr = []
        while True:
            btarr.insert(0,number % 128)
            if number < 128:
                break
            number //= 128
        btarr[-1] +=128
        return struct.pack("{0}B".format(len(btarr)),*btarr)
    btarr = []
    for number in lst:
        btarr.append(encode_n(number))
    return b''.join(btarr)

def decode(stream):
    numbers = []
    temp = 0
    bytestream = struct.unpack("{0}B".format(len(stream)),stream)
    for byte in bytestream:
        if byte < 128:
            temp = 128*temp + byte
        else:
            temp = 128*temp + (byte - 128)
            numbers.append(temp)
            temp = 0
    return numbers
            


In [2]:
from sortedcontainers import SortedDict
import pickle

In [3]:
with open('table_with_pos.pickle','rb') as stream:
    dictionary = pickle.load(stream)

## Без сжатия

In [4]:
index = list(dictionary.values())
print(sys.getsizeof(index[0].keys()))
print(sys.getsizeof(index[15].values()))
keys1 = []
values1 = []
for idx in range(len(index)):
    keys1.append(index[idx].keys())
    v = []
    for key in index[idx].keys():
        v.append(index[idx][key])
    values1.append(v)

48
48


In [23]:
sys.getsizeof(keys1[0])
list(keys1[0])

[854, 17744, 30267, 37488]

In [68]:
sys.getsizeof(values1)

11007832

In [5]:
with open('uncompressed_values.pickle','wb') as stream:
    pickle.dump(values1,stream)

In [18]:
keys = []
values = []
for idx in range(len(index)):
    keys.append(encode(list(index[idx].keys())))
    v = []
    for key in index[idx].keys():
        v.append(encode(index[idx][key]))
    values.append(v)

        

In [22]:
len(keys[0])

11

In [17]:
values[0]

[b'\x11\xcf', b'\x04\xa6', b'+\xca', b'\x94']

In [11]:
sys.getsizeof(values[0])

96

In [15]:
with open('compressed_keys.pickle','wb') as stream:
    pickle.dump(keys,stream)

In [24]:
with open('compressed_values.pickle','wb') as stream:
    pickle.dump(values,stream)

In [1]:
import os

## Размер координатных индексов (положения слов в документах)

In [13]:
a = os.stat('uncompressed_values.pickle').st_size

## Размер файла после сжатия

In [14]:
b = os.stat('compressed_values.pickle').st_size

## Процент сжатия

In [15]:
((a-b)/a) * 100

14.374900311523891

## Размер координатных блоков 

In [8]:
a = os.stat('table_coord.pickle').st_size

## Размер после сжатия

In [9]:
b = os.stat('compressed_keys.pickle').st_size

## Процент сжатия

In [11]:
((a-b)/a) * 100

2.987577424394823